In [56]:
# Import the libraries we will use
import pandas as pd
import numpy as np
# from scipy.stats.stats import pearsonr
#import matplotlib.pyplot as plt
#import matplotlib as mpl
#from sklearn.metrics import mean_squared_error
# import plotly.offline as pl
# import plotly.graph_objs as go
# pl.init_notebook_mode(connected=True)
#from datetime import datetime
import matplotlib.pyplot as plt
import plotly.offline as pl
import plotly.graph_objs as go
pl.init_notebook_mode(connected=True)

In [69]:
#exchange = ('bitstamp_btc_usd','cex_btc_usd','exmo_btc_usd','bitfinex_btc_usd','kraken_btc_usd', 'gdax_btc_usd')
exchange = 'bitfinex_btc_usd'
#time = ('10s','20s','30s')
#alpha_all = (1.5,2,2.5,3)
alpha_all = 1,1.5,2,3,5,10
#n = 0 # Exchange number
S = 0.00 # SPREAD > S
time_period = '120s' # '60s' '30min' -- prediction time
#alpha = 5 # K > alpha
beta = 20 # K < beta
a = -0.0 # BID[t]-BID[t-1]> a*SPREAD[t-1]
LimitUPall = 3,5,10,20
LimitDown = 3
TheBestProfit = 0

In [70]:
print(exchange)
print(time_period)
# Read ask-bid price from the file
series_exch = pd.read_csv(exchange+'.csv', index_col=0)
series_exch.index = pd.to_datetime(series_exch.index, unit='ms') # Time converting and +3 hour
#series_exch.tail()
series_period = series_exch.resample(time_period).mean()
series_max = series_exch.resample(time_period).max()
series_min = series_exch.resample(time_period).min()
series_first = series_exch.resample(time_period).first()
series_last = series_exch.resample(time_period).last()
#series_period = series_exch
series_period['spread'] = series_period['ask'] - series_period['bid']
#series_period.tail()
N = len(series_period.index)
BID = series_period['bid']
BIDmax = series_max['bid']
BIDmin = series_min['bid']
BidLast = series_last['bid']
AskFirst = series_first['ask']
BidFirst = series_first['bid']
ASK = series_period['ask']
SPREAD = series_period['spread']
step = 1+5 # 1 back and 8 forward
for  LimitUP in LimitUPall:  
    for alpha in alpha_all:
        print(S,'$')
        print('alpha = ',alpha)
        print('LimitUP = ',LimitUP)
        print('LimitDown = ',LimitDown)
        K = np.ones(N-step)
        Bidsuper = np.zeros(N-step)
        Bidworse = np.zeros(N-step)
        #ASKnow = np.zeros(N-step)
        Profit = np.zeros(N-step)
        Loss = np.zeros(N-step)
        BidFinal = np.zeros(N-step)
        SPREADbefore = np.zeros(N-step)
        for t in range(1,N-step):
            if ASK[t]-ASK[t-1]>0 and BID[t]-BID[t-1]>a*SPREAD[t-1]:
                K[t] = SPREAD[t]/SPREAD[t-1]
            Bidsuper[t] = np.nanmax([BIDmax[t+1],BIDmax[t+2],BIDmax[t+3],BIDmax[t+4],BIDmax[t+5]])
            Bidworse[t] = np.nanmin([BIDmin[t+1],BIDmin[t+2],BIDmin[t+3],BIDmin[t+4],BIDmin[t+5]])
            Loss[t] = BidLast[t+step-1] - AskFirst[t+1]
            if Bidworse[t]<BidFirst[t+1]-LimitDown:
                Loss[t] = BidFirst[t+1]-LimitDown-AskFirst[t+1]
            Profit[t] = Bidsuper[t] - AskFirst[t+1]
            SPREADbefore[t] = SPREAD[t-1]
        # Creating dataframe and log-transform y
        data = {'Loss': Loss, 'Profit': Profit, 'SPREAD_before': SPREADbefore, 'K': K}
        df_data = pd.DataFrame(data=data)
        df_data = df_data[df_data['K']>alpha].copy() # K>alpha
        df_data = df_data[df_data['K']<beta].copy() # K<beta
        df_data = df_data[df_data['SPREAD_before']>S].copy()
        df_data_plus = df_data[df_data['Profit']>=LimitUP].copy()
        df_data_minus = df_data[df_data['Profit']<LimitUP].copy()
        print('Количество:', len(df_data))
        #print('Отношение:', len(df_data_plus)/len(df_data))
        #print('Средний доход:', np.nanmean(df_data_plus['Profit'].values))
        print('Доход:', LimitUP*len(df_data_plus))
        print('Убыток:', np.nansum(df_data_minus['Loss'].values))
        TheCurrentProfit = LimitUP*len(df_data_plus) + np.nansum(df_data_minus['Loss'].values)
        if TheCurrentProfit > TheBestProfit:
            TheBestProfit = TheCurrentProfit
        print('Прибыль:', TheCurrentProfit)
        #print(df_data)
        #df_data_plus['Profit'].reset_index(drop=True, inplace=True)
        #df_data_plus['Profit'].plot()
        #plt.show()
        print(' ')
print(TheBestProfit)

bitfinex_btc_usd
120s
0.0 $
alpha =  0.99
LimitUP =  3
LimitDown =  3
Количество: 1576
Доход: 2796
Убыток: -1893.7000000000426
Прибыль: 902.2999999999574
 
0.0 $
alpha =  1.5
LimitUP =  3
LimitDown =  3
Количество: 219
Доход: 354
Убыток: -310.2999999999938
Прибыль: 43.700000000006185
 
0.0 $
alpha =  2
LimitUP =  3
LimitDown =  3
Количество: 148
Доход: 237
Убыток: -221.6999999999898
Прибыль: 15.300000000010186
 
0.0 $
alpha =  3
LimitUP =  3
LimitDown =  3
Количество: 68
Доход: 114
Убыток: -94.29999999999563
Прибыль: 19.700000000004366
 
0.0 $
alpha =  5
LimitUP =  3
LimitDown =  3
Количество: 26
Доход: 51
Убыток: -31.399999999997817
Прибыль: 19.600000000002183
 
0.0 $
alpha =  10
LimitUP =  3
LimitDown =  3
Количество: 3
Доход: 6
Убыток: -4.5
Прибыль: 1.5
 
0.0 $
alpha =  0.99
LimitUP =  5
LimitDown =  3
Количество: 1576
Доход: 4170
Убыток: -2044.6000000000513
Прибыль: 2125.3999999999487
 
0.0 $
alpha =  1.5
LimitUP =  5
LimitDown =  3
Количество: 219
Доход: 520
Убыток: -328.599999999

In [ ]:
# series_exch['spread'] = series_exch['ask'] - series_exch['bid']
#series_exch[series_exch['spread']>=0.02]

In [ ]:
# bid_usd = go.Scatter(
#             x = series_period.index,
#             y = series_period['bid']-9200,
#             name = 'BID Exchange rate, '+time_period,
#             #mode = 'markers',
#             line = dict(
#             color = 'mediumseagreen',
#             width = 2,
#             #dash = 'dash' # dash options include 'dash', 'dot', and 'dashdot'
#             )            
# )
# ask_usd = go.Scatter(
#             x = series_period.index,
#             y = series_period['ask']-9200,
#             name = 'ASK Exchange rate, '+time_period,
#             #mode = 'markers',
#             line = dict(
#             color = 'lightred',
#             width = 2,
#             #dash = 'dash' # dash options include 'dash', 'dot', and 'dashdot'
#             )            
# )
# data = [bid_usd, ask_usd]
# layout = dict(title = 'BTC ASK price, '+exchange,
#               xaxis = dict(title = 'Time'),
#               yaxis = dict(title = 'USD'),
#               #paper_bgcolor='rgb(255,255,255)',
#               #plot_bgcolor='rgb(229,229,229)',
#               )    
# fig = dict(data=data, layout=layout)
# pl.iplot(fig, filename='styled-line')